Final Project: finding confounding variables

Idan Hermelin

Noam Biton

In [439]:
import pandas as pd

# Primary dataset
gdp_growth_df = pd.read_csv('GDPs\gdp_per_capita_growth.csv') 
#We choose to use the gdp per capita growth because it is a better indicator of the economic growth of a country than only gdp
gdp_ppp_df = pd.read_csv('GDPs\gdp_ppp_per_capita.csv')
gdp_growth_df.head()

# Auxiliary datasets
happiness_2015_df = pd.read_csv('happiness_score/2015.csv')
happiness_2016_df = pd.read_csv('happiness_score/2016.csv')
happiness_2017_df = pd.read_csv('happiness_score/2017.csv')
happiness_2018_df = pd.read_csv('happiness_score/2018.csv')
happiness_2019_df = pd.read_csv('happiness_score/2019.csv')

print("columns of the datasets:")
print(happiness_2015_df.columns)
print(happiness_2016_df.columns)
print(happiness_2017_df.columns)
print(happiness_2018_df.columns)
print(happiness_2019_df.columns)

columns of the datasets:
Index(['Country', 'Region', 'Happiness Rank', 'Happiness Score',
       'Standard Error', 'Economy (GDP per Capita)', 'Family',
       'Health (Life Expectancy)', 'Freedom', 'Trust (Government Corruption)',
       'Generosity', 'Dystopia Residual'],
      dtype='object')
Index(['Country', 'Region', 'Happiness Rank', 'Happiness Score',
       'Lower Confidence Interval', 'Upper Confidence Interval',
       'Economy (GDP per Capita)', 'Family', 'Health (Life Expectancy)',
       'Freedom', 'Trust (Government Corruption)', 'Generosity',
       'Dystopia Residual'],
      dtype='object')
Index(['Country', 'Happiness.Rank', 'Happiness Score', 'Whisker.high',
       'Whisker.low', 'Economy (GDP per Capita)', 'Family',
       'Health (Life Expectancy)', 'Freedom', 'Generosity',
       'Trust..Government.Corruption.', 'Dystopia.Residual'],
      dtype='object')
Index(['Overall rank', 'Country', 'Happiness Score',
       'Economy (GDP per Capita)', 'Social support',
   

In [440]:
gdp_growth_df.rename(columns={"Country Name": "Country"}, inplace=True)

In [441]:
#change happiness columns to have the same name:
happiness_2017_df.rename(columns={"Happiness.Score": "Happiness Score", "Health..Life.Expectancy.":"Health (Life Expectancy)", "Economy..GDP.per.Capita.":"Economy (GDP per Capita)"}, inplace=True)
happiness_2018_df.rename(columns={"Score": "Happiness Score", "GDP per capita":"Economy (GDP per Capita)"}, inplace=True)
happiness_2019_df.rename(columns={"Score": "Happiness Score", "GDP per capita":"Economy (GDP per Capita)"}, inplace=True)
happiness_2017_df.to_csv('happiness_score/2017.csv', index=False)
happiness_2018_df.to_csv('happiness_score/2018.csv', index=False)
happiness_2019_df.to_csv('happiness_score/2019.csv', index=False)

In [442]:
#change country column to have the same name:
print(happiness_2019_df.columns)
happiness_2018_df.rename(columns={"Country or region": "Country", "Freedom to make life choices": "Freedom", "Healthy life expectancy":"Health (Life Expectancy)"}, inplace=True)
happiness_2019_df.rename( columns={"Country or region": "Country", "Freedom to make life choices": "Freedom", "Healthy..life expectancy":"Health (Life Expectancy)"}, inplace=True)
happiness_2018_df.to_csv('happiness_score/2018.csv', index=False)
happiness_2019_df.to_csv('happiness_score/2019.csv', index=False)

print(happiness_2019_df.columns)

Index(['Overall rank', 'Country', 'Happiness Score',
       'Economy (GDP per Capita)', 'Social support',
       'Health (Life Expectancy)', 'Freedom', 'Generosity',
       'Perceptions of corruption'],
      dtype='object')
Index(['Overall rank', 'Country', 'Happiness Score',
       'Economy (GDP per Capita)', 'Social support',
       'Health (Life Expectancy)', 'Freedom', 'Generosity',
       'Perceptions of corruption'],
      dtype='object')


Functions for creating dfs of GDP trends

In [443]:
def merge_happiness(gdp_growth_df, happiness_df):
    return gdp_growth_df.merge(happiness_df, on=["Country"], how="left")

In [444]:
def GDP_happiness_df(year1, year2, merged_df):
    last_10_years_df = [col for col in merged_df.columns if (col.isdigit() and int(col) >= year1 and int(col) <= year2)]
    merged_df = merged_df[["Country", "Happiness Score", "Freedom", "Health (Life Expectancy)", "Economy (GDP per Capita)"] + last_10_years_df]
    merged_df = merged_df.dropna()
    return merged_df

In [445]:
def calc_trend_growth(merged_df, year1, year2):
    last_10_years_cols = [col for col in merged_df.columns if (col.isdigit() and int(col) >= year1 and int(col) <= year2)]
    merged_df['GDP Growth Trend Over 10 Years'] = merged_df[last_10_years_cols].mean(axis=1, skipna=True)
    return merged_df

Create the dfs that contains the trends in the GDP:

In [446]:
# Create merged datasets for gdp per capita
merged_2015_to_2019_dfs = []
for year in range(2015, 2020):  
    happiness_df = pd.read_csv(f'happiness_score/{year}.csv')
    merged_df = merge_happiness(gdp_growth_df, happiness_df)
    merged_df = GDP_happiness_df(year-8, year, merged_df)
    merged_df = calc_trend_growth(merged_df, year-8, year)
    merged_2015_to_2019_dfs.append(merged_df)

In [447]:
merged_2015_to_2019_dfs[0].head()

,Country,Happiness Score,Freedom,Health (Life Expectancy),Economy (GDP per Capita),2007,2008,2009,2010,2011,2012,2013,2014,2015,GDP Growth Trend Over 10 Years
2,Afghanistan,3.575,0.23414,0.30335,0.31982,11.022787,1.594215,18.515419,11.264091,-2.681061,8.974865,1.974166,-0.665291,-1.622857,5.375148
4,Angola,4.033,0.10384,0.16683,0.75778,9.890049,7.116848,-2.808600,0.640263,-0.220843,4.706519,1.291994,1.219881,-2.468737,2.151930
5,Albania,4.959,0.35733,0.81325,0.87867,6.787229,8.328108,4.053140,4.223084,2.821642,1.584873,1.187234,1.985388,2.516827,3.720836
8,United Arab Emirates,6.901,0.64157,0.80925,1.42727,-11.345653,-10.208646,-15.151225,-5.914931,2.186344,2.257099,4.409938,4.225677,4.507818,-2.781509
9,Argentina,6.574,0.44974,0.78723,1.05351,7.924211,3.029501,-6.854225,9.300123,4.788683,-2.145284,1.265685,-3.578581,1.629664,1.706642


In [448]:
# Normalize the GDP Growth Trend Over 10 Years column so that it is between -1 and 1:
# Use rubast scaler to normalize the data
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
for i in range(0, len(merged_2015_to_2019_dfs)):
    merged_2015_to_2019_dfs[i]['GDP Growth Trend Over 10 Years'] = scaler.fit_transform(merged_2015_to_2019_dfs[i][['GDP Growth Trend Over 10 Years']])
print(merged_2015_to_2019_dfs[0].head())


                Country  Happiness Score  Freedom  Health (Life Expectancy)  \
2           Afghanistan            3.575  0.23414                   0.30335   
4                Angola            4.033  0.10384                   0.16683   
5               Albania            4.959  0.35733                   0.81325   
8  United Arab Emirates            6.901  0.64157                   0.80925   
9             Argentina            6.574  0.44974                   0.78723   

   Economy (GDP per Capita)       2007       2008       2009       2010  \
2                   0.31982  11.022787   1.594215  18.515419  11.264091   
4                   0.75778   9.890049   7.116848  -2.808600   0.640263   
5                   0.87867   6.787229   8.328108   4.053140   4.223084   
8                   1.42727 -11.345653 -10.208646 -15.151225  -5.914931   
9                   1.05351   7.924211   3.029501  -6.854225   9.300123   

       2011      2012      2013      2014      2015  \
2 -2.681061  8.9748

In [449]:
#check the mean value of normelized GDP growth:
for i in range(0, len(merged_2015_to_2019_dfs)):
    print(merged_2015_to_2019_dfs[i]['GDP Growth Trend Over 10 Years'].mean())

0.04781470745784676
0.08757269783037869
0.09757704185442709
0.09324346861541954
0.0713953380328811


These values are more representing the real growth in GDP because in most countries, the expected GDP growth over the last 20 years has averaged around 2–4% annually for developed economies and 5–7%

In [450]:
#!pip install statsmodels

In [451]:
import statsmodels.api as sm

def rank_confounders(df, treatment, outcome, confounders):
    rankings = []
    for confounder in confounders:
        # Model without confounder
        model1 = sm.OLS(df[outcome], sm.add_constant(df[[treatment]])).fit()
        r2_without = model1.rsquared

        # Model with confounder
        model2 = sm.OLS(df[outcome], sm.add_constant(df[[treatment, confounder]])).fit()
        r2_with = model2.rsquared

        r2_change = r2_with - r2_without
        rankings.append((confounder, r2_change))

    rankings.sort(key=lambda x: x[1], reverse=True)
    return pd.DataFrame(rankings, columns=["Confounder", "R² Change"])

In [ ]:
treatment_options = ["Freedom", "Health (Life Expectancy)"]
outcome = "Happiness Score"
confounders = ["GDP Growth Trend Over 10 Years", ""]
year = 2015
for df in merged_2015_to_2019_dfs:
    #print(df)
    for treatment in treatment_options:
        print(f"for year {year} the treatment is {treatment} and the outcome is {outcome}")
        result_df = rank_confounders(df, treatment, outcome, confounders)
        print(result_df)
    year += 1

for year 2015 the treatment is Freedom and the outcome is Happiness Score
                       Confounder  R² Change
0        Economy (GDP per Capita)   0.389142
1  GDP Growth Trend Over 10 Years   0.068523
for year 2015 the treatment is Health (Life Expectancy) and the outcome is Happiness Score
                       Confounder  R² Change
0        Economy (GDP per Capita)   0.106111
1  GDP Growth Trend Over 10 Years   0.009551
for year 2016 the treatment is Freedom and the outcome is Happiness Score
                       Confounder  R² Change
0        Economy (GDP per Capita)   0.406603
1  GDP Growth Trend Over 10 Years   0.112808
for year 2016 the treatment is Health (Life Expectancy) and the outcome is Happiness Score
                       Confounder  R² Change
0        Economy (GDP per Capita)   0.082132
1  GDP Growth Trend Over 10 Years   0.005816
for year 2017 the treatment is Freedom and the outcome is Happiness Score
                       Confounder  R² Change
0        Ec

Do the same calculation about GDP ppp:

In [453]:
# Create merged datasets for gdp ppp per capita
# merged_2015_to_2019_ppp_dfs = []
# for year in range(2015, 2020):  
#     happiness_df = pd.read_csv(f'happiness_score/{year}.csv')
#     merged_df = merge_happiness(gdp_ppp_df, happiness_df)
#     merged_df = GDP_happiness_df(year-10, year, merged_df)
#     merged_df = calc_trend_growth(merged_df, year-10, year)
#     merged_2015_to_2019_ppp_dfs.append(merged_df)

Anlysis with countries population dataset

In [454]:
countries_df = pd.read_csv('countries\world_popilation.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'countries\\world_popilation.csv'